# 函数式编程
函数式编程就是一种抽象程度很高的编程范式  
函数式编程的一个特点就是，允许把函数本身作为参数传入另一个函数，还允许返回一个函数
## 高阶函数 Higher-order function
- 函数名是个指针
- 变量名也是个指针，所以变量名可以指向函数名  
- `abs(-10)`是函数调用，而`abs`是函数本身  
- 结论：函数本身也可以赋值给变量，即：变量可以指向函数。就是函数指针

In [1]:
f=abs
f(-1)

1

- 函数名也是变量,可以改变build-in function-name的指向
- 要恢复abs函数，请重启Python交互环境

In [2]:
# abs=10
# abs(-1)

### 传入函数
一个函数就可以接收另一个函数作为参数，这种函数就称之为高阶函数。

In [3]:
func=abs
def add(x,y,f):
    return f(x)+f(y)

add(-10,5,func)

15

### map/reduce
map()函数接收两个参数:
- 一个是函数，
- 一个是Iterable

作用：
- map将**传入的函数**依次作用到**序列的每个元素**，并把结果作为新的Iterator返回。
- map的返回值是Iterator迭代器，而且是惰性的，要用list算出每一个元素

举例：
- 想把[1,2,3,4],每一个数据变成$x^2$

In [4]:
def func(x):
    return x**2
list(map(func,[1,2,3,4]))

[1, 4, 9, 16]

map()作为高阶函数，事实上它把运算规则抽象了，可以一眼看出语句的作用。  
因此，我们不但可以计算简单的f(x)=x2，还可以计算任意复杂的函数，比如，把这个list所有数字转为字符串：

In [5]:
list(map(str, [1, 2, 3, 4, 5, 6, 7, 8, 9]))

['1', '2', '3', '4', '5', '6', '7', '8', '9']

reduce() 的参数：
- 函数
- Iterable

作用：  
reduce把结果继续和序列的下一个元素做累积计算，其效果就是：
> reduce(f, [x1, x2, x3, x4]) = f(f(f(x1, x2), x3), x4)

In [6]:
from functools import reduce
def fn(x,y):
    return x*10 +y
def char2num(s):
    digits={'0':0,'1':1,'2':2,'3':3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
    return digits[s]

reduce(fn,map(char2num,'123456'))

123456

In [7]:
## 整理成一个函数
from functools import reduce

DIGITS = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}

def str2int(s):
    def fn(x, y):
        return x * 10 + y
    def char2num(s):
        return DIGITS[s]
    return reduce(fn, map(char2num, s))

## 使用lambda函数，简化
def char2num(s):
    return DIGITS[s]

def str2int(s):
    return reduce(lambda x, y: x * 10 + y, map(char2num, s))

str2int('12345')

12345

### filter
Python内建的filter()函数用于过滤序列。
参数：
- 函数
- Iterable

In [8]:
##取出奇数的数字
def is_odd(n):
    return n % 2 == 1

list(filter(is_odd, [1, 2, 4, 5, 6, 9, 10, 15]))

[1, 5, 9, 15]

In [9]:
##提取出非空的字符
def not_empty(s):
    return s and s.strip()

list(filter(not_empty, ['ABC ', '', 'B ', 'C', '  ']))

['ABC ', 'B ', 'C']

### 排序算法
sorted()也是一个高阶函数。用sorted()排序的关键在于实现一个映射函数。
此外，sorted()函数也是一个高阶函数，它还可以接收一个key函数来实现自定义的排序，例如按绝对值大小排序：

In [10]:
sorted([36, 5, -12, 9, -21])

[-21, -12, 5, 9, 36]

In [11]:
sorted([36, 5, -12, 9, -21], key=abs)

[5, 9, -12, -21, 36]

字符串排序

In [12]:
sorted(['bob', 'about', 'Zoo', 'Credit'])

['Credit', 'Zoo', 'about', 'bob']

In [13]:
sorted(['bob', 'about', 'Zoo', 'Credit'], key=str.lower)

['about', 'bob', 'Credit', 'Zoo']

In [14]:
sorted(['bob', 'about', 'Zoo', 'Credit'], key=str.lower, reverse=True)

['Zoo', 'Credit', 'bob', 'about']

In [15]:
L = [('Bob', 75), ('Adam', 92), ('Bart', 66), ('Lisa', 88)]
sorted(L,key=lambda x:x[1],reverse=True)

[('Adam', 92), ('Lisa', 88), ('Bob', 75), ('Bart', 66)]

## 返回函数
函数作为返回值
高阶函数除了可以接受函数作为参数外，还可以把函数作为结果值返回。
- 但是，如果不需要立刻求和，而是在后面的代码中，根据需要再计算怎么办？可以不返回求和的结果，而是返回求和的函数：
- 请再注意一点，当我们调用lazy_sum()时，每次调用都会返回一个新的函数，即使传入相同的参数：

In [16]:
def lazy_sum(*args):
    def sum():
        ax = 0
        for n in args:
            ax = ax + n
        return ax
    return sum

In [17]:
f = lazy_sum(1, 3, 5, 7, 9)
f

<function __main__.lazy_sum.<locals>.sum()>

In [18]:
f()

25

### 闭包
注意到返回的函数在其定义内部引用了局部变量args，所以，当一个函数返回了一个函数后，其内部的局部变量还被新函数引用，所以，闭包用起来简单，实现起来可不容易。  
 返回闭包时牢记一点：返回函数不要引用任何循环变量，或者后续会发生变化的变量。

## 匿名函数
匿名函数`lambda x: x * x`实际上就是：  
``def f(x):
    return x * x``
    
限制：
- 就是只能有一个表达式，不用写return，返回值就是该表达式的结果。

好处：
- 因为函数没有名字，不必担心函数名冲突。
- 匿名函数也是一个函数对象，也可以把匿名函数赋值给一个变量，再利用变量来调用该函数


In [19]:
list(map(lambda x: x * x, [1, 2, 3, 4, 5, 6, 7, 8, 9]))

[1, 4, 9, 16, 25, 36, 49, 64, 81]

In [20]:
f = lambda x: x * x
f,f(5)

(<function __main__.<lambda>(x)>, 25)

## 装饰器
由于函数也是一个对象，而且函数对象可以被赋值给变量，所以，通过变量也能调用该函数。


In [21]:
def now():
    print('2015-3-25')
f = now
f(),now.__name__,f.__name__

2015-3-25


(None, 'now', 'now')

现在，假设我们要增强now()函数的功能，比如，在函数调用前后自动打印日志，但又不希望修改now()函数的定义，这种在代码运行期间动态增加功能的方式，称之为“装饰器”（Decorator）。  
本质上，decorator就是一个返回函数的高阶函数。所以，我们要定义一个能打印日志的decorator，可以定义如下：

In [22]:
def log(func):
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper


观察上面的log，因为它是一个decorator，所以接受一个函数作为参数，并返回一个函数。我们要借助Python的@语法，把decorator置于函数的定义处：

In [23]:
@log
def now():
    print('2015-3-25')
now()

call now():
2015-3-25


把@log放到now()函数的定义处，相当于执行了语句：
`now = log(now)`
由于log()是一个decorator，返回一个函数，所以，原来的now()函数仍然存在，只是现在同名的now变量指向了新的函数，于是调用now()将执行新函数，即在log()函数中返回的wrapper()函数。

注意：
`now.__name__` 变了

In [24]:
now.__name__

'wrapper'

In [25]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper

In [26]:
@log
def now():
    print('2015-3-25')
now()
now.__name__

call now():
2015-3-25


'now'

## 偏函数
Python的functools模块提供了很多有用的功能，其中一个就是偏函数（Partial function）  
functools.partial就是帮助我们创建一个偏函数的，不需要我们自己定义int2()，可以直接使用下面的代码创建一个新的函数int2：  
把一个函数的某些参数给固定住（也就是设置默认值），返回一个新的函数，调用这个新函数会更简单。

In [27]:
import functools
int2 = functools.partial(int, base=2)
int2('1000000')

64

下面两种表达式是等价的。
- 理解`kw = { 'base': 2 }   **kw`其实就是`base=2`
- 理解`args=[1,2,3]    *args`其实就是`1,2,3`

In [28]:
kw = { 'base': 2 }
int('10010', **kw)

18

In [29]:
int2 = functools.partial(int, base=2)
int2('10010')

18